In [20]:
import pandas as pd
import numpy as np
import torch
import json

from tqdm import tqdm
from torch import nn
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from timeit import default_timer as timer
from os import walk
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_curve, auc, brier_score_loss

In [21]:
ROOT_DATA = '../../data'
HUMAN_JSON_FILE_NAME = 'human.jsonl'
HUMAN_JSON_PATH = f'{ROOT_DATA}/raw/{HUMAN_JSON_FILE_NAME}'
MODELS_JSON_FOLDER_PATH = f'{ROOT_DATA}/raw/machines'
BASELINE_MODELS_FOLDER_PATH = f'{ROOT_DATA}/baseline/models'
BASELINE_MODELS_RESULTS_FOLDER_PATH = f'{ROOT_DATA}/baseline/results'

In [22]:
BATCH_SIZE = 32
LSTM_UNITS = 256
LSTM_LAYERS = 5
EMBEDDING_SIZE = 512

In [23]:
TEST_SET_FRACTION = 0.3

In [24]:
df = pd.read_json(path_or_buf=HUMAN_JSON_PATH, lines=True)
df['text_index'] = df.index
df['is_llm'] = 0
df['dataset_name'] = Path(HUMAN_JSON_FILE_NAME).stem

In [25]:
dir_path, dir_names, file_names = next(walk(MODELS_JSON_FOLDER_PATH))

for file_name in file_names:
    temp_df = pd.read_json(path_or_buf=f'{MODELS_JSON_FOLDER_PATH}/{file_name}', lines=True)
    temp_df['text_index'] = temp_df.index
    temp_df['is_llm'] = 1
    temp_df['dataset_name'] = Path(file_name).stem

    df = pd.concat([df, temp_df], ignore_index=True)

df.drop(labels=['id'], inplace=True, axis='columns')

In [26]:
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [27]:
df['tokenized_text'] = tokenizer(list(df['text'].to_list()))['input_ids']

Token indices sequence length is longer than the specified maximum sequence length for this model (843 > 512). Running this sequence through the model will result in indexing errors


In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [29]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, layers_num, device, output_size=1, dropout=0):
        super().__init__()

        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.layers_num = layers_num
        self.output_size= output_size
        self.dropout = dropout
        self.device = device

        self.embed = nn.Embedding(self.vocab_size, self.embedding_size, device=self.device)

        self.lstm = nn.LSTM(
            input_size=self.embedding_size,
            hidden_size=self.hidden_size,
            num_layers=self.layers_num,
            batch_first=True,
            dropout=self.dropout,
            device=self.device
        )

        self.fc = nn.Linear(
            self.hidden_size,
            self.output_size
        )

    def forward(self, X, lengths):
        embeddings = self.embed(X)

        seq_output, (h_n, c_n) = self.lstm(embeddings)

        out = seq_output.sum(dim=1).div(lengths.float().unsqueeze(dim=1))
        logits = self.fc(out)
        return logits

In [30]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.size

    def __getitem__(self, index):
        return (
            self.X[index],
            self.y[index]
        )

In [31]:
def collate_fn(batch):
  # We want to sort the batch by seq length,
  # in order to make the computation more efficient
  batch = sorted(batch, key=lambda x: len(x[0]), reverse=True)

  inputs = [torch.LongTensor(x[0]).to(device) for x in batch]
  padded_input = nn.utils.rnn.pad_sequence(inputs, batch_first=True)

  lengths = torch.LongTensor([len(x[0]) for x in batch]).to(device)

  y = torch.FloatTensor(np.array([x[1] for x in batch])).reshape(-1, 1).to(device)

  return padded_input, lengths, y

In [32]:
def calculate_accuracy(y_true, y_hat):
    correct_pred = torch.eq(torch.sigmoid(y_hat).round(), y_true).sum().item()
    return (correct_pred / len(y_hat)) * 100

def calculate_f1(y_true, y_hat):
    y_pred = torch.sigmoid(y_hat).round()
    return f1_score(y_true, y_pred)

def calculate_brier(y_true, y_hat):
    y_prob = torch.sigmoid(y_hat)
    return brier_score_loss(y_true, y_prob)

def calculate_auc(y_true, y_hat):
    y_prob = torch.sigmoid(y_hat)

    false_positive_rates, true_positive_rates, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(false_positive_rates, true_positive_rates)

    return roc_auc, false_positive_rates, true_positive_rates

In [33]:
def test_step(model, dataloader, loss_fn, device):
    model.eval()

    all_y_true = []
    all_y_hat = []

    test_loss = 0
    steps = 0

    with torch.inference_mode():
        for X, lengths, y in dataloader:

            X, y = X.to(device), y.to(device)

            y_hat = model(X, lengths)

            all_y_true.extend(y)
            all_y_hat.extend(y_hat)

            loss = loss_fn(y_hat, y)
            test_loss += loss.item()

            steps += 1

        all_y_true = torch.FloatTensor(all_y_true)
        all_y_hat = torch.FloatTensor(all_y_hat)

        test_accuracy = calculate_accuracy(all_y_true, all_y_hat)
        test_f1 = calculate_f1(all_y_true, all_y_hat)
        test_brier = calculate_brier(all_y_true, all_y_hat)
        test_auc_tuple = calculate_auc(all_y_true, all_y_hat)

    return test_loss / steps, test_accuracy, test_f1, test_brier, test_auc_tuple

def test(model, loss_fn, device, test_df):
    test_dataset_full = TextDataset(test_df['tokenized_text'], test_df['is_llm'])
    test_dataloader_full = DataLoader(
            test_dataset_full,
            batch_size=BATCH_SIZE,
            shuffle=False,
            drop_last=False,
            collate_fn=collate_fn
        )

    start_time = timer()

    test_loss, test_acc, test_f1, test_brier, test_auc_tuple = test_step(
        model,
        test_dataloader_full,
        loss_fn,
        device
    )

    end_time = timer()

    results_formatted = {
        "test_loss": test_loss,
        "test_acc": test_acc,
        "test_f1": test_f1,
        "test_brier": test_brier,
        "test_auc_tuple": test_auc_tuple
    }
        
    print(
        f"test_loss: {test_loss:.4f} | "
        f"test_acc: {test_acc:.4f} | "
        f"test_f1: {test_f1:.4f} | "
        f"test_brier: {test_brier:.4f} | "
        f"time: {(end_time-start_time):.4f}"
    )
    
    return results_formatted

In [34]:
human_df = df.loc[df['is_llm'] == 0]
human_train_df, human_test_df = train_test_split(human_df, test_size=TEST_SET_FRACTION, random_state=69)

In [35]:
class json_serialize(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [36]:
final_results = []

for llm_name in df['dataset_name'].unique():
    if llm_name == Path(HUMAN_JSON_FILE_NAME).stem:
        continue
    
    llm_df = df.loc[df['dataset_name'] == llm_name]
    llm_df_train, llm_df_test = train_test_split(llm_df, test_size=TEST_SET_FRACTION, random_state=69)
    test_df = pd.concat([human_test_df, llm_df_test], ignore_index=True)    

    model_path = f'{BASELINE_MODELS_FOLDER_PATH}/{llm_name}.pt'

    model = RNN(tokenizer.vocab_size, EMBEDDING_SIZE, LSTM_UNITS, LSTM_LAYERS, device, dropout=0.6).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))

    loss_fn = nn.BCEWithLogitsLoss()

    print(f'Testing {llm_name} against itself...')

    results_formatted = test(model, loss_fn, device, test_df)

    with open(f'{BASELINE_MODELS_RESULTS_FOLDER_PATH}/{llm_name}_test_metrics.json', 'w', encoding='utf-8') as f:
        json.dump(results_formatted, f, ensure_ascii=False, indent=4, cls=json_serialize)

    print(f'Finished testing {llm_name} against itself.')
    

Testing alpaca-7b against itself...
test_loss: 0.3260 | test_acc: 94.4954 | test_f1: 0.9439 | test_brier: 0.0480 | time: 21.0661
Finished testing alpaca-7b against itself.
Testing bigscience-bloomz-7b1 against itself...
test_loss: 0.5942 | test_acc: 84.7095 | test_f1: 0.8270 | test_brier: 0.1172 | time: 22.2123
Finished testing bigscience-bloomz-7b1 against itself.
Testing chavinlo-alpaca-13b against itself...
test_loss: 0.2721 | test_acc: 96.1774 | test_f1: 0.9610 | test_brier: 0.0340 | time: 23.1426
Finished testing chavinlo-alpaca-13b against itself.
Testing gemini-pro against itself...
test_loss: 0.1784 | test_acc: 93.8838 | test_f1: 0.9373 | test_brier: 0.0480 | time: 26.7833
Finished testing gemini-pro against itself.
Testing gpt-3.5-turbo-0125 against itself...
test_loss: 0.3837 | test_acc: 92.0489 | test_f1: 0.9205 | test_brier: 0.0624 | time: 24.0087
Finished testing gpt-3.5-turbo-0125 against itself.
Testing gpt-4-turbo-preview against itself...
test_loss: 0.0531 | test_acc: 